In [398]:
import capytaine as cpt
import numpy as np
import matplotlib.pyplot as plt
from capytaine.bem.airy_waves import airy_waves_potential, airy_waves_velocity, froude_krylov_force



In [399]:
#%%writefile pwa_utils.py
def generate_body(xyz,r):
    mesh1 = cpt.meshes.predefined.mesh_sphere(radius=r,center=(xyz[0],xyz[1],xyz[2]))
    body = cpt.FloatingBody(mesh1)
    body.add_translation_dof(name='Heave')
    body = body.immersed_part()
    body.name = f'{xyz[0]}_{xyz[1]}_{xyz[2]}'
    return body


def get_results(problems):
    results = [solver.solve(pb, keep_details = True) for pb in sorted(problems)]
    return results


#calculate angle theta_ij from centre of one body to other
def theta_ij(X,Y): 
    x1,y1= X[0],X[1]
    x2,y2 = Y[0], Y[1]
    if x2==x1:
        theta = np.pi/2
    else:
        theta = np.arctan((y2-y1)/(x2-x1))
    return theta


#step 2
def phi_j_star(phi_ij,theta,X,Y,z,k):
    
    '''phi_ij is the vector of all the effect at that body from all other bodies'''
    x,y = X[0],X[1]
    xj,yj = Y[0],Y[1]
    res = phi_ij *np.exp(k*z)*np.exp(1j*k*(x-xj)*np.cos(theta) + (y-yj)*np.sin(theta))
    return res


def get_phistarj_sum(phi_starj,xyzees):
    xyz_phi = {xyz:[] for xyz in xyzees}
    for k,v in phi_starj.items():
        for s,m in v.items():
            xyz_phi[k].append(m)
    xyz_phi = {k:sum(v) for k,v in xyz_phi.items()}
    return xyz_phi

p = 0
omega = 1
rho = 850 # density of our special material
wave_amp = 1
wave_num =  1.0/9.81

The method is based upon an idea due to Simon 3
originally devised in connection with the theory of arrays
of wave-power devices. A diverging wave scattered from
one cylinder is replaced by a plane wave of appropriate
amplitude in the neighbourhood of another cylinder. Once
the amplitude and phase of the equivalent plane wave have
been determined, the problem reduces to summing the
effects of plane waves on any given cylinder

#### 1. Initialize the bodies and define diffraction and radiation problems and solve

In [415]:
#xyzees = {(0,0,0),(5,5,0),(10,10,0),(15,15,0)}

def xyzees(x1,x2,x3,x4,y1,y2,y3,y4) :
    d1 = np.sqrt((x4 - x1)**2 + (y4 - y1)**2)
    d2 = np.sqrt((x3 - x1)**2 + (y3 - y1)**2)
    d3 = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    d4 = np.sqrt((x3 - x2)**2 + (y3 - y2)**2)
    d5 = np.sqrt((x4 - x3)**2 + (y4 - y3)**2)
    d6 = np.sqrt((x4 - x2)**2 + (y4 - y2)**2)
    xyzees = {(x1,y1,0),(x2,y2,0),(x3,y3,0),(x4,y4,0)}
    return xyzees

bodies = [generate_body(xyz) for xyz in xyzees ]

neighbors = {(0,0,0):[(5,5,0),(10,10,0),(15,15,0)],  #so bad..need to write a funky func for it
            (10,10,0):[(0,0,0),(5,5,0),(15,15,0)],
             (5,5,0):[(0,0,0),(10,10,0),(15,15,0)],
             (15,15,0):[(0,0,0),(10,10,0),(5,5,0)]     
            }
loc_bodies = {body:xyz for xyz,body in zip(xyzees,bodies)}
loc_to_body = {xyz:body for xyz,body in zip(xyzees,bodies)}
solver = cpt.BEMSolver()


diff_problems = {body:cpt.DiffractionProblem(body=body, sea_bottom=-np.infty,
                                      omega=omega, wave_direction=0.) for body in bodies}

diff_loc= {generate_body(loc):loc for loc in xyzees }

rad_problems = {body: cpt.RadiationProblem(body=body, sea_bottom=-np.infty,
                                      omega=omega) for body in bodies}

diff_results = {body.name:solver.solve(problem) for body,problem in diff_problems.items()}
rad_results = {body.name:solver.solve(problem) for body,problem in rad_problems.items()}


### 2. Get the Incident potentials, diffraction potentials, radiation potentials for each body at their own location and the location of other bodies
Currently only incident potential can be generated at any other location from capytaine. But diffraction and radiation are available for the mesh
However, according to simon(1982) we can approximate the impact of outgoing waves from a body on all other bodies by an incident plane wave of appropriately chosen amplitude.


#### find the location of neighbors to each floating body

In [416]:
body_neighbors_locs = {body:neighbors.get(loc_bodies.get(body)) for body in bodies}
body_neighbors_locs

{FloatingBody(mesh=sphere_1062, dofs={Heave}, name=10_10_0): [(0, 0, 0),
  (5, 5, 0),
  (15, 15, 0)],
 FloatingBody(mesh=sphere_1068, dofs={Heave}, name=0_0_0): [(5, 5, 0),
  (10, 10, 0),
  (15, 15, 0)],
 FloatingBody(mesh=sphere_1074, dofs={Heave}, name=5_5_0): [(0, 0, 0),
  (10, 10, 0),
  (15, 15, 0)],
 FloatingBody(mesh=sphere_1080, dofs={Heave}, name=15_15_0): [(0, 0, 0),
  (10, 10, 0),
  (5, 5, 0)]}

###  Get the potential of a body to other location

In [422]:
body_potential_at_neighbors = {body:(dict(zip(body_neighbors_locs[body], 
                                      airy_waves_potential(np.array(body_neighbors_locs[body]),diff_problems[body])))) for body in bodies}
body_potential_at_neighbors

{FloatingBody(mesh=sphere_1062, dofs={Heave}, name=10_10_0): {(0,
   0,
   0): -9.81j,
  (5, 5, 0): (4.78631304488035-8.563136541969198j),
  (15, 15, 0): (9.801453837976808-0.409393040988372j)},
 FloatingBody(mesh=sphere_1068, dofs={Heave}, name=0_0_0): {(5,
   5,
   0): (4.78631304488035-8.563136541969198j),
  (10, 10, 0): (8.355933157169986-5.139502025771294j),
  (15, 15, 0): (9.801453837976808-0.409393040988372j)},
 FloatingBody(mesh=sphere_1074, dofs={Heave}, name=5_5_0): {(0, 0, 0): -9.81j,
  (10, 10, 0): (8.355933157169986-5.139502025771294j),
  (15, 15, 0): (9.801453837976808-0.409393040988372j)},
 FloatingBody(mesh=sphere_1080, dofs={Heave}, name=15_15_0): {(0,
   0,
   0): -9.81j,
  (10, 10, 0): (8.355933157169986-5.139502025771294j),
  (5, 5, 0): (4.78631304488035-8.563136541969198j)}}

### $\phi_{ij}$ Get the all other potential at each location/body

In [423]:
# def get_all_other_phi(body_potential_at_neighbors):
all_other_phi_each_loc = {xyz:{loc_bodies.get(d):k.get(xyz,0) for d,k in body_potential_at_neighbors.items()} for xyz in xyzees}
all_other_phi_each_loc

{(10, 10, 0): {(10, 10, 0): 0,
  (0, 0, 0): (8.355933157169986-5.139502025771294j),
  (5, 5, 0): (8.355933157169986-5.139502025771294j),
  (15, 15, 0): (8.355933157169986-5.139502025771294j)},
 (0, 0, 0): {(10, 10, 0): -9.81j,
  (0, 0, 0): 0,
  (5, 5, 0): -9.81j,
  (15, 15, 0): -9.81j},
 (5, 5, 0): {(10, 10, 0): (4.78631304488035-8.563136541969198j),
  (0, 0, 0): (4.78631304488035-8.563136541969198j),
  (5, 5, 0): 0,
  (15, 15, 0): (4.78631304488035-8.563136541969198j)},
 (15, 15, 0): {(10, 10, 0): (9.801453837976808-0.409393040988372j),
  (0, 0, 0): (9.801453837976808-0.409393040988372j),
  (5, 5, 0): (9.801453837976808-0.409393040988372j),
  (15, 15, 0): 0}}

### $\phi_j^*$ get the total effect of each bodies

In [424]:
#step 2

thetas = {k:{s:theta_ij(k,s) for s,m in v.items()} for k,v in all_other_phi_each_loc.items()}
thetas

{(10, 10, 0): {(10, 10, 0): 1.5707963267948966,
  (0, 0, 0): 0.7853981633974483,
  (5, 5, 0): 0.7853981633974483,
  (15, 15, 0): 0.7853981633974483},
 (0, 0, 0): {(10, 10, 0): 0.7853981633974483,
  (0, 0, 0): 1.5707963267948966,
  (5, 5, 0): 0.7853981633974483,
  (15, 15, 0): 0.7853981633974483},
 (5, 5, 0): {(10, 10, 0): 0.7853981633974483,
  (0, 0, 0): 0.7853981633974483,
  (5, 5, 0): 1.5707963267948966,
  (15, 15, 0): 0.7853981633974483},
 (15, 15, 0): {(10, 10, 0): 0.7853981633974483,
  (0, 0, 0): 0.7853981633974483,
  (5, 5, 0): 0.7853981633974483,
  (15, 15, 0): 1.5707963267948966}}

In [425]:
z = 0
phi_starj = {xyz:{nbros:phi_j_star(all_other_phi_each_loc[xyz][nbros],thetas[xyz][nbros],nbros,xyz,z,wave_num) for nbros in neighbors} for xyz in xyzees}
phi_starj

{(10, 10, 0): {(0, 0, 0): (0.0024508233898185737-0.00796327665513729j),
  (10, 10, 0): 0j,
  (5, 5, 0): (0.1750534680839285-0.2260355652628886j),
  (15, 15, 0): (330.49066021169506-63.912059009019785j)},
 (0, 0, 0): {(0, 0, 0): 0j,
  (10, 10, 0): (7623.078615263059-8677.500134439013j),
  (5, 5, 0): (118.70667937207196-314.988177629559j),
  (15, 15, 0): (349771.3422179954-186381.20333507293j)},
 (5, 5, 0): {(0, 0, 0): (0.042520874079842966-0.2827149955687011j),
  (10, 10, 0): (257.3020925054289-217.03562142252642j),
  (5, 5, 0): 0j,
  (15, 15, 0): (10887.940381557542-3855.2678772451914j)},
 (15, 15, 0): {(0, 0, 0): (0.00010514678977658235-0.00021887121014465565j),
  (10, 10, 0): (0.26308702588915334-0.11189732016775249j),
  (5, 5, 0): (0.006024614705580329-0.005755373838739027j),
  (15, 15, 0): 0j}}

### Get the Total potential on the body surface

In [426]:


new_excitation = get_phistarj_sum(phi_starj,xyzees)
new_excitation     

{(10, 10, 0): (330.6681645031688-64.14605785093781j),
 (0, 0, 0): (357513.1275126305-195373.6916471415j),
 (5, 5, 0): (11145.284994937052-4072.5862136632863j),
 (15, 15, 0): (0.26921678738451027-0.11787156521663618j)}

Having computed this effect at each body in the array we then compute the contribution of all the bodies as isolated ( as in step 1) induced by new excitation

#### Get the all other potential at each location/body with new excitation
If new excitation is the potential and we have this new excitation **at** each location, how do we go from getting the contribution of this $\phi$ to other location?

- Logical thing seem here is to apply the phi_starj function to the other location with xyz and get new excitation until the 2*times the number of bodies as suggested in the paper. ok! let's just do that

- First thing is to somehow calculate the potential to the neighbor as with the airywave potential earlier but this time its new excitation from lsat time.

- Calculate new `body_potential_at_neighbors` from earlier to loop it back in a while loop. The potential at other location this time is not going to be the airywave potential calculation but just the  calculation function `phi_j_star` but keep new potential at each neigbors in the format `body_potential_at_neighbors` 



In [427]:
body_potential_at_neighbors = {body:{nbros : phi_j_star(new_excitation[xyz],thetas[loc_bodies[body]][nbros],nbros,xyz,z,wave_num)
                                              for nbros in neighbors} for body in bodies
                                          }

KeyError: (7, 11, 0)

In [428]:
                           
body_potential_at_neighbors

{FloatingBody(mesh=sphere_1062, dofs={Heave}, name=10_10_0): {(0,
   0,
   0): -9.81j,
  (5, 5, 0): (4.78631304488035-8.563136541969198j),
  (15, 15, 0): (9.801453837976808-0.409393040988372j)},
 FloatingBody(mesh=sphere_1068, dofs={Heave}, name=0_0_0): {(5,
   5,
   0): (4.78631304488035-8.563136541969198j),
  (10, 10, 0): (8.355933157169986-5.139502025771294j),
  (15, 15, 0): (9.801453837976808-0.409393040988372j)},
 FloatingBody(mesh=sphere_1074, dofs={Heave}, name=5_5_0): {(0, 0, 0): -9.81j,
  (10, 10, 0): (8.355933157169986-5.139502025771294j),
  (15, 15, 0): (9.801453837976808-0.409393040988372j)},
 FloatingBody(mesh=sphere_1080, dofs={Heave}, name=15_15_0): {(0,
   0,
   0): -9.81j,
  (10, 10, 0): (8.355933157169986-5.139502025771294j),
  (5, 5, 0): (4.78631304488035-8.563136541969198j)}}

### Now we basically while loop the whole thing until the max-iteration

In [429]:
N_bodies = 4
max_iteration = 2*N_bodies #(dead or alive lol)

body_potential_at_neighbors = {body:(dict(zip(body_neighbors_locs[body], 
                                      airy_waves_potential(np.array(body_neighbors_locs[body]),diff_problems[body])))) for body in bodies}
iterate = 0
while iterate<max_iteration:
    # def get_all_other_phi(body_potential_at_neighbors):
    all_other_phi_each_loc = {xyz:{loc_bodies.get(d):k.get(xyz,0) for d,k in body_potential_at_neighbors.items()} for xyz in xyzees}
    thetas = {k:{s:theta_ij(k,s) for s,m in v.items()} for k,v in all_other_phi_each_loc.items()}
    phi_starj = {xyz:{nbros:phi_j_star(all_other_phi_each_loc[xyz][nbros],thetas[xyz][nbros],nbros,xyz,z,wave_num) for nbros in neighbors} for xyz in xyzees}
    
    new_excitation = get_phistarj_sum(phi_starj,xyzees)
    body_potential_at_neighbors = {body:{nbros : phi_j_star(new_excitation[xyz],thetas[loc_bodies[body]][nbros],nbros,xyz,z,wave_num)
                                              for nbros in neighbors} for xyz,body in loc_to_body.items()}
    
    print(body_potential_at_neighbors)
    print("/n")
    
    iterate+=1

new_potential = get_phistarj_sum(phi_starj,xyzees)
    

{FloatingBody(mesh=sphere_1062, dofs={Heave}, name=10_10_0): {(0, 0, 0): (0.1750355630550878-0.22628437392475026j), (10, 10, 0): (330.6681645031688-64.14605785093781j), (5, 5, 0): (8.358368975482286-5.147707656097085j), (15, 15, 0): (11393.591035602245+1941.617728607751j)}, FloatingBody(mesh=sphere_1068, dofs={Heave}, name=0_0_0): {(0, 0, 0): (357513.1275126305-195373.6916471415j), (10, 10, 0): (468060063.48979473+104994438.47945282j), (5, 5, 0): (13843483.250350958-1947115.8909015274j), (15, 15, 0): (13758394013.80172+9035052265.182096j)}, FloatingBody(mesh=sphere_1074, dofs={Heave}, name=5_5_0): {(0, 0, 0): (262.08665957685287-225.60690485881582j), (10, 10, 0): (407143.3430782609+4098.191918786222j), (5, 5, 0): (11145.284994937052-4072.5862136632863j), (15, 15, 0): (13023328.941500422+5058258.54121705j)}, FloatingBody(mesh=sphere_1080, dofs={Heave}, name=15_15_0): {(0, 0, 0): (5.58883363034053e-07-7.252885026054282e-06j), (10, 10, 0): (0.006130379530982646-0.0059812944929139135j), (5

In [430]:
body_potential_at_neighbors 

{FloatingBody(mesh=sphere_1062, dofs={Heave}, name=10_10_0): {(0,
   0,
   0): (506305.3417946687-1368630.3598404734j),
  (10, 10, 0): (1511380377.144763-817195179.1727451j),
  (5, 5, 0): (32818120.513810415-37818609.27847617j),
  (15, 15, 0): (58417275924.60412-7950649787.596264j)},
 FloatingBody(mesh=sphere_1068, dofs={Heave}, name=0_0_0): {(0,
   0,
   0): (1511380377.144763-817195179.1727451j),
  (10, 10, 0): (1971921153996.94+451596346116.8248j),
  (5, 5, 0): (58417275924.60412-7950649787.596264j),
  (15, 15, 0): (57851615500773.05+38361643451786.64j)},
 FloatingBody(mesh=sphere_1074, dofs={Heave}, name=5_5_0): {(0,
   0,
   0): (32818120.513810415-37818609.27847617j),
  (10, 10, 0): (58417275924.60412-7950649787.596264j),
  (5, 5, 0): (1511380377.144763-817195179.1727451j),
  (15, 15, 0): (1971921153996.94+451596346116.8248j)},
 FloatingBody(mesh=sphere_1080, dofs={Heave}, name=15_15_0): {(0,
   0,
   0): (-258.4658606914054-42527.2539013676j),
  (10, 10, 0): (32818120.513810415-

In [431]:
new_potential[(10, 10, 0)]

(1511380377.144763-817195179.1727451j)

In [432]:
# function to input new potential from plane-wave approximation
def solve(body,new_potential,keep_details=True):
        """Solve the linear potential flow problem.
        Parameters
        ----------
        problem: LinearPotentialFlowProblem
            the problem to be solved
        keep_details: bool, optional
            if True, store the sources and the potential on the floating body in the output object
            (default: True)
        Returns
        -------
        LinearPotentialFlowResult
            an object storing the problem data and its results
        """

        potential = new_potential
        rho = 1000
        new_pressure = rho * new_potential
        # Actually, for diffraction problems: pressure over jω
        #           for radiation problems:   pressure over -ω²
        # The correction is done in `store_force` in the `result` object.

        new_forces = body.integrate_pressure(new_pressure)

#         if not keep_details:
#             result = problem.make_results_container(new_forces)
#         else:
#             result = problem.make_results_container(new_forces, sources, new_potential, new_pressure)


        return new_forces
    

    


In [433]:
new_results = [solve(diff_prob, new_potential[loc]) for diff_prob,loc in loc_diff.items()]
new_results
    

KeyError: (7, 11, 0)

Somehow keep track of the delta and use the last one before it got less than 10e-2??

In [434]:
%%writefile interaction_phi.py
def interaction_phi(radius):
    '''takes radius = 0.5m for now..look at the default xyz for now.. and create its neighbor and provide it..
    for now lets go with this'''
    def generate_body(xyz):
        mesh1 = cpt.meshes.predefined.mesh_sphere(radius=radius,center=(xyz[0],xyz[1],xyz[2]))
        body = cpt.FloatingBody(mesh1)
        body.add_translation_dof(name='Heave')
        body = body.immersed_part()
        body.name = f'{xyz[0]}_{xyz[1]}_{xyz[2]}'
        return body


    def get_results(problems):
        results = [solver.solve(pb, keep_details = True) for pb in sorted(problems)]
        return results


    #calculate angle theta_ij from centre of one body to other
    def theta_ij(X,Y): 
        x1,y1= X[0],X[1]
        x2,y2 = Y[0], Y[1]
        if x2==x1:
            theta = np.pi/2
        else:
            theta = np.arctan((y2-y1)/(x2-x1))
        return theta


    #step 2
    def phi_j_star(phi_ij,theta,X,Y,z,k):

        '''phi_ij is the vector of all the effect at that body from all other bodies'''
        x,y = X[0],X[1]
        xj,yj = Y[0],Y[1]
        res = phi_ij *np.exp(k*z)*np.exp(1j*k*(x-xj)*np.cos(theta) + (y-yj)*np.sin(theta))
        return res


    def get_phistarj_sum(phi_starj,xyzees):
        xyz_phi = {xyz:[] for xyz in xyzees}
        for k,v in phi_starj.items():
            for s,m in v.items():
                xyz_phi[k].append(m)
        xyz_phi = {k:sum(v) for k,v in xyz_phi.items()}
        return xyz_phi
    
    
    xyzees = {(0,0,0),(10,10,0),(5,5,0),(7,11,0)}
    
    bodies = [generate_body(xyz) for xyz in xyzees ]

    neighbors = {(0,0,0):[(10,10,0),(5,5,0),(7,11,0),(0,0,0)],  #so bad..need to write a funky func for it
                (10,10,0):[(0,0,0),(5,5,0),(7,11,0),(10,10,0)],
                 (5,5,0):[(0,0,0),(10,10,0),(7,11,0),(5,5,0)],
                 (7,11,0):[(0,0,0),(10,10,0),(5,5,0),(7,11,0)]     
                }
    loc_bodies = {body:xyz for xyz,body in zip(xyzees,bodies)}
    loc_to_body = {xyz:body for xyz,body in zip(xyzees,bodies)}
    solver = cpt.BEMSolver()


    diff_problems = {body:cpt.DiffractionProblem(body=body, sea_bottom=-np.infty,
                                          omega=omega, wave_direction=0.) for body in bodies}

    rad_problems = {body: cpt.RadiationProblem(body=body, sea_bottom=-np.infty,
                                          omega=omega) for body in bodies}

    diff_results = {body.name:solver.solve(problem) for body,problem in diff_problems.items()}
    rad_results = {body.name:solver.solve(problem) for body,problem in rad_problems.items()}
    body_neighbors_locs = {body:neighbors.get(loc_bodies.get(body)) for body in bodies}
    body_potential_at_neighbors = {body:(dict(zip(body_neighbors_locs[body], 
                                      airy_waves_potential(np.array(body_neighbors_locs[body]),diff_problems[body])))) for body in bodies}
    
    # def get_all_other_phi(body_potential_at_neighbors):
    all_other_phi_each_loc = {xyz:{loc_bodies.get(d):k.get(xyz,0) for d,k in body_potential_at_neighbors.items()} for xyz in xyzees}
    thetas = {k:{s:theta_ij(k,s) for s,m in v.items()} for k,v in all_other_phi_each_loc.items()}
    z = 0
    phi_starj = {xyz:{nbros:phi_j_star(all_other_phi_each_loc[xyz][nbros],thetas[xyz][nbros],nbros,xyz,z,wave_num) for nbros in neighbors} for xyz in xyzees}
    new_excitation = get_phistarj_sum(phi_starj,xyzees)
    body_potential_at_neighbors = {body:{nbros : phi_j_star(new_excitation[xyz],thetas[loc_bodies[body]][nbros],nbros,xyz,z,wave_num)
                                              for nbros in neighbors} for body in bodies }
    
    
    N_bodies = 4
    max_iteration = 2*N_bodies #(dead or alive lol)

    body_potential_at_neighbors = {body:(dict(zip(body_neighbors_locs[body], 
                                      airy_waves_potential(np.array(body_neighbors_locs[body]),diff_problems[body])))) for body in bodies}
    iterate = 0
    while iterate<max_iteration:
        # def get_all_other_phi(body_potential_at_neighbors):
        all_other_phi_each_loc = {xyz:{loc_bodies.get(d):k.get(xyz,0) for d,k in body_potential_at_neighbors.items()} for xyz in xyzees}
        thetas = {k:{s:theta_ij(k,s) for s,m in v.items()} for k,v in all_other_phi_each_loc.items()}
        phi_starj = {xyz:{nbros:phi_j_star(all_other_phi_each_loc[xyz][nbros],thetas[xyz][nbros],nbros,xyz,z,wave_num) for nbros in neighbors} for xyz in xyzees}

        new_excitation = get_phistarj_sum(phi_starj,xyzees)
        body_potential_at_neighbors = {body:{nbros : phi_j_star(new_excitation[xyz],thetas[loc_bodies[body]][nbros],nbros,xyz,z,wave_num)
                                                  for nbros in neighbors} for body in bodies}

        print(body_potential_at_neighbors)
        print("/n")

        iterate+=1
    new_potential = get_phistarj_sum(phi_starj,xyzees)
    return new_potential


    
    




Overwriting interaction_phi.py
